# Day 3 -- Python for Researchers

## Today's Goals:
   * Use the pandas library
   * Understand what a dataframe and series are
   * Learn how to execute basic data cleaning
   * Learn some key functions for data analysis 

**Introduction to Pandas**

Pandas is the industry standard data analysis library. It lets us convert raw datasets (typically a .csv file) into something called a *dataframe*. A dataframe looks like a spreadsheet, but is actually optimized to allow you to handle large datasets quickly and efficiently. We can use pandas to clean, analyze, and even visualize our data. 

In pandas you can work with two kinds of objects -- a dataframe, or a 2D array (something with rows and columns) or a series, a 1D array (or just a  singular column). Both of these objects comes with different built-in functions that we will explore throughout today's lesson. 

Pandas is a really powerful tool for researchers. Oftentimes, the way you really learn to master it is by looking at examples and reading documentation. They have a ["10 Minutes to Pandas"](https://pandas.pydata.org/docs/user_guide/10min.html) guide we recommend you return to after this week 

#### Section 1: Our first dataset 

To start our exploration of pandas, we're going to use a Film Permits dataset from the [NYC Open Data site](https://data.cityofnewyork.us/City-Government/Film-Permits/tg4x-b46p/about_data). That dataset is already in our Day 3 folder. Please note: when you gather a dataset online, it typically comes with it's own internal logic -- especially goverment datasets -- so you will want to study any documentation they provide to better undersatnd your data. 

In order to use pandas, you always have to import it. The common way to do this is to "import pandas as pd" -- pd just is a shorter amout of letters to type in. 

To create your dataframe, you run the code below. Our dataframe is being stored in the variable permits_df. Take a second to study the output, what can we learn from it? 

In [ ]:
import pandas as pd
permits_df = pd.read_csv("Film_Permits_20250518.csv")

permits_df

**Learning about our dataset**

We're going to use four techniques to learn about our dataset:

  * info() -- tells you how many rows and columns there are; the names of each column; the different kinds of datatype contained in the columns; and how many non-null values there are in each column
  * head() -- shows by default the top five rows, you can add an optional parameter (a number) to show more
  * tail() -- same as head(), except it shows you the bottom rows
  * .dtypes -- tells you what each column's data type is



In [ ]:
permits_df.info()

In [ ]:
permits_df.head()

In [ ]:
permits_df.tail()

Note that there are seven datatypes in pandas, some of which correspond to the datatypes in Python:

* object (equivalent to a Python string)
* int64 (integer)
* float64 (float aka decimal)
* bool (Boolean, True/False values)
* datetime64 (date and time values)
* timedelta[ns] (differences between two values)
* category (finite list of text values, corresponds to categorical variables in statistics)

It's important to know what types of data you are manipulating, because they all come with unique abilities and limitations.

What kind of datatypes does our dataframe contain? 

In [ ]:
permits_df.dtypes

We can also access specific information in our dataframe. The most frequent way you will do this is to isolate a column. 

The preferred syntax for this is to use brackets and input the column name as strings. You can isolate only one at a time, 
or isolate multiple by writing them in a list.

In [ ]:
#isolating one column
permits_df["Borough"]

In [ ]:
#isolating multiple columns
permits_df[["EventID", "Borough", "Category"]]

If you want to isolate very specific information, you can use .iloc and .loc

 * You use .loc when dealing with column names (labels), e.g., below isolates the first six rows in the EventType column (label), notice it uses inclusive slicing
 * You use .iloc when you want to index a specific row and column (iloc stands of “integer location”),
   e.g., below isolates the same thing using index numbers and uses exclusive slicing


In [ ]:
#example of loc

permits_df.loc[:5, "EventType"]

In [ ]:
#example of iloc

permits_df.iloc[:5]

**Preparing our dataset for analysis**

Here's our research question: *How many of each film permit type was distributed in each borough?* Because this dataset was produced by NYC, it's pretty clean. We just want to drop the columns that don't help us answer our research question so that we can save memory. It's not a huge dataset, so this isn't necessary, but is useful to know how to do. 

Here are all our columns (I turned them into a list for easy reading).

In [ ]:
list(permits_df.columns)

We probably don't care about a number of those partiular columns, so let's drop them.

It's best practice to make a copy of your dataframe before you start changing it, so we'll start with that: 

In [9]:
permits_df_copy = permits_df.copy()

Then we can drop several of those columns using drop(). Here's some info about what we're doing:
 * drop() elimates columns
 * You have to tell it what column names you want to drop, this can be passed as a list 
 * You need to tell it you're dropping a column (not a row) which is axis = 1 (axis = 0 is for rows)
 * inplace = True makes sure this affects the dataframe; otherwise, it would create a copy of the dataframe where the change happens 

I've dropped two columns below, try adding another column name to the list that we don't need! 

In [11]:
permits_df_copy.drop(["CommunityBoard(s)", "PolicePrecinct(s)"], axis = 1, inplace = True)

**Doing the analysis**

We need to take the next four steps to answer our RQ:
  * isolate the two columns that have our information (EventType and Borough)
  * use value_counts() which helps us aggregate values in columns
  * convert this into it's own dataframe using reset_index()
  * sort the columns for easier viewing using sort_values()

In [ ]:
#isolate our two columns:

rq_df = permits_df_copy[["EventType", "Borough"]]
rq_df

In [ ]:
#count the values
#this LOOKS like it is a dataframe, but it's actually a series

rq_df = rq_df.value_counts()
rq_df

In [ ]:
#reset the index to transform it into a dataframe

rq_df = rq_df.reset_index()
rq_df.columns = ["Event", "Borough", "Count"] #names columns
rq_df

In [ ]:
rq_df.sort_values("Event", inplace = True, ascending = False) 
rq_df

It's your turn! 

Try to answer this research question: *Which borough distributed the most permits?*

In [ ]:
#your answer

#### Section 2: Our Second Dataset 

You can find our second dataset [here](https://drive.google.com/file/d/1RgcK9VPCIxVMDZabQAJhWCp7z0LD_Im3/view?usp=sharing). It has information about the top 1000 ranking movies on IMDB. Please download that and bring it into your working directory.

Oftentimes online movie ratings have a recency bias -- that is, the more recently a movie was made, the more likely it is to have a high review because people have just seen it. That leads us to ask a few questions: *How many different years are represented in this dataset? Which years have the highest number of movies associated with them? And of those years, are most of them after 2000?* 

In [ ]:
import pandas as pd

film_df = pd.read_csv("imdb_top_1000 (1).csv")
film_df.info()

**Cleaning data three different ways**

There's actually an error in our dataset that is going to create an issue for us. Do you notice that the Release_Year column is an object datatype? You wouldn't expect that because it should *in theory* all be numbers. What I found is that one singular row has the string "PG" in it instead of a year. 

In [ ]:
#proof that we have one singular string in our Released_Year column

for i in film_df["Released_Year"].to_list():
    if i.isalpha():
        print(i)

PG


*Using loc[]*

I can use loc[] (which we learned above) to isolate this row because I know the value (pandas calls this a label). Remember: loc[] lets us access rows and columns by the label! This then shows me the index. I can then use the index to drop that row from my dataset.     

In [ ]:
film_df.loc[film_df["Released_Year"] == "PG"]

In [ ]:
#notice that we only have 999 rows now compared to before!

film_df_copy = film_df.drop(966)
film_df_copy.info()

*Using isin()*

This looks for a list of values in the row, and if they ARE present, it drops them. Notice that we're isolating the appropriate column, using .isin() to pass a list of values (in this case, we only had one value!), and only keep the rows that do NOT have those values in them.


In [ ]:

film_df_copy_2 = film_df[film_df["Released_Year"].isin(["PG"]) == False]
film_df_copy_2.info()

*Flexible solution with notna()*

There is an inherent limitation to the previous two solutions: if we have more than one row with alphabetical characters, and those are different, it becomes quite tedious! This is called *hard coding*, where you create a very inflexible bit of code that only works for one singluar situation. Generally, you want to build flexible code that can handle multiple situations. 

Thus, the better, and more flexible, method would be to simply drop the rows if they contained alphabetical characters. This produces the same effect as above, but would work with ANY row that contained alphabetical characters. 

Here's a breakdown:
  * We isolated the Released_Year column and try to convert each value to a number using to_numeric(). If that value cannot be converted, it becomes NaN. This means Not A Number and is pandas version of "None" or "Null".
  * the function notna() in pandas returns a True Boolean for any value that isn't NAN
  * this construction automatically keeps only the rows where notna() returned a True Boolean 

In [ ]:
film_df = film_df[pd.to_numeric(film_df["Released_Year"], errors="coerce").notna()]
film_df.info()

Importantly, that column is STILL listed as an object datatype, we have to take the final step to convert it into an int64 datatype.

In [95]:
film_df = film_df.astype({"Released_Year": "int64"})

Now we can answer our RQ! It turns out that there are 100 years of movies represented in this dataset, but 17 out of the 20 top-rated movies come from after 2000. So I would definitely say it has a recency bias!

In [ ]:
#creating new dataframe
year_aggregate = film_df["Released_Year"].value_counts()
year_aggregate = year_aggregate.reset_index()
year_aggregate.columns = ["Release Year", "Number of Movies"]

year_aggregate

In [ ]:
#isolate top 20
top_20 = year_aggregate.head(20)
top_20

In [101]:
#boolean masking
condition_1 = top_20["Release Year"] >= 2000
final_df = top_20[condition_1]
final_df.count()

Release Year        17
Number of Movies    17
dtype: int64

Now that we've done ALL that, we can also evaluate which release year has the highest rating on average to see if there's a bias that way. There doesn't seem to be one!

Here's what groupby() does:
  * It groups rows in a dataframe based on the values of one or more columns. In this case, we're grouping the release year column by the IMDB rating. 
  * Notice the syntax, we are grouping the rows in the Released_Year column and we're taking the mean (average) IMDB rating for each. 

In [85]:
avg_rating = film_df.groupby("Released_Year")["IMDB_Rating"].mean()

In [ ]:
avg_rating_df = avg_rating.reset_index()
avg_rating_df.columns = ["Year", "Avg_Rating"]

avg_rating_df.sort_values("Avg_Rating", inplace = True, ascending = False) 
avg_rating_df.head(20)

If you understand this recency bias, you could isolate only the rows that come from before 2000 and do your analysis excluding more recent movies.

In [ ]:
film_df.loc[film_df["Released_Year"] < 2000]

We could similarly choose to only look at movies from a particular decade:

In [ ]:
column = film_df["Released_Year"]

film_df.loc[(column >= 1980) & (column <= 1989)]

#### Section 3: Your turn!

You are going to continue to use the IMDB dataset to answer the question *what is the average IMDB rating of the top 10 directors (by frequency) represented in the dataset?*

  To answer this:

  *   First, isolate the top 10 directors (as a hint: Alfred Hitchcock is number one)
  *   Like we did above, convert that information into a dataframe.
  *   Isolate the column with the directors in it and convert it into a list using [tolist()](https://www.geeksforgeeks.org/python-pandas-series-tolist/).
  *   I want you to adapt the syntax that we learned above to strip PG out of the dataframe. You want to create a new copy of the dataframe and strip all directors out of it except those in the list we just created.
  *   Finally, you can use groupby() to figure out what the average IMDB score is for those director's movies.
